In [ ]:
!pip install --upgrade crewai crewai-tools

In [ ]:

from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool
from langchain_google_genai import ChatGoogleGenerativeAI
import os

# 🔐 Replace with your actual keys
GEMINI_API_KEY = 'your_gemini_api_key'
SERPER_API_KEY = 'your_serper_api_key'

os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY
os.environ["SERPER_API_KEY"] = SERPER_API_KEY


In [ ]:

llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.7)


In [ ]:

search_tool = SerperDevTool()


In [ ]:

destination = "Delhi"

researcher = Agent(
    role="Travel Researcher",
    goal=f"Find historical sites, hotels near transport, and weather for {destination}.",
    verbose=True,
    allow_delegation=False,
    backstory="Expert travel researcher with real-time web access.",
    tools=[search_tool],
    llm=llm
)

budget_planner = Agent(
    role="Budget Planner",
    goal=f"Estimate travel and hotel budget for 3 days in {destination}.",
    verbose=True,
    allow_delegation=False,
    backstory="A seasoned travel budget planner.",
    llm=llm
)

itinerary_planner = Agent(
    role="Itinerary Planner",
    goal=f"Create a 3-day travel plan based on research and budget for {destination}.",
    verbose=True,
    allow_delegation=False,
    backstory="Expert at organizing activities into optimal itineraries.",
    llm=llm
)

research_task = Task(
    description=f"Find historical sites, weather, and hotels near transport for {destination}.",
    agent=researcher
)

budget_task = Task(
    description=f"Create a realistic 3-day travel budget for {destination}, including food, hotel, and transport.",
    agent=budget_planner
)

itinerary_task = Task(
    description=f"Using research and budget, create a 3-day itinerary for {destination}.",
    agent=itinerary_planner
)


In [ ]:

crew = Crew(
    agents=[researcher, budget_planner, itinerary_planner],
    tasks=[research_task, budget_task, itinerary_task],
    verbose=True,
    process=Process.sequential
)

result = crew.kickoff()
print("\n🎉 Final Itinerary Output:\n")
print(result)
